# Grand Challenge - Genome Assembly

CSCI 3104 - Algorithm, Spring 2019

Team Name: Ce Qiu, Borui Yu

-----

### Algorithm Design

In [1]:
# this function calculating the overlaping length from read1 to read2
# example: read1 = "AAAATCTCCAGTGCCCAAGA"
#          read2 =            "TGCCCAAGACCACGGGCGCT"
# overlap(read1, read2) will return 9
def overlap(read1, read2):
    j = 0   # index in read2
    res = 0 # overlap result
    for i in range(len(read1)):
        if read1[i] == read2[j]:
            # if has overlap
            j += 1
            res += 1
        else: # if the continuous overlap break
            j = 0
            res = 0
            if read1[i] == read2[j]:
                j += 1
                res += 1
    return res

In [20]:
# this function read the .fq file and create a graph basing on all the reads
# example: read1 = "AAAATCTCCAGTGCCCAAGA"
#          read2 =            "TGCCCAAGACCACGGGCGCT"
# graph_generator(filename) will return [[0, 9], [0, 0]]
# thus, we can create a graph with: read[i] is node[i], weight(v, u) is overlap(v, u)
# in other words, graph[0][1] is overlap value from read0 to read1 is 9
def graph_generator(filename):
    reads = [] # empty list for saving reads
    count = 3  # file example see below:
    # line 1, count = 3: @random ......
    # line 2, count = 4: read we need
    # line 3, count = 5: +
    # line 4, count = 6: some repeated letter
    # line 5, count = 7: @random ......
    # line 6, count = 8: read we need
    # ......
    # when count % 4 == 0, we collect the reads
    for line in open(filename):
        if count % 4 == 0:
            reads.append(line.strip())
        count += 1
    # create the graph by using list:
    # example, we have 3 reads: r1, r2, r3
    # we create a matrix represent the overlap value from rows to column
    # overlap = [[       0      , overlap(r1,r2), overlap(r1,r3)],
    #            [overlap(r2,r1),        0      , overlap(r2,r3)],
    #            [overlap(r3,r1), overlap(r3,r2),        0      ]]
    graph = []
    for r1 in reads:     # for each read in the set
        sub_graph = []
        for r2 in reads: # finding the weight to all others
            if r1 == r2: # ignore self-loop
                sub_graph.append(0)
            else:
                o = overlap(r1, r2)
                if o > 0: # this can be change, at least how many overlap we want
                    sub_graph.append(o)
                else: # less than minimum overlap requirement, ignore
                    sub_graph.append(0)
        graph.append(sub_graph)
    return graph, reads

In [3]:
# finding the index of the maximum overlap value in graph
def find_max(graph):
    row, column, max_v = 0, 0, 0
    for i in range(len(graph)):
        if max(graph[i]) > max_v:
            max_v = max(graph[i])
            row = i
            column = graph[i].index(max_v)
    return row, column, max_v

In [4]:
# this function is combine the sub_graph in to result graph
def combine(res, sub_graph):
    if res == []: # if there is no sub-graph in res
        res.append(sub_graph)
    else:
        check = False
        for i in res:
            if sub_graph[len(sub_graph) - 1] == i[0]:
                if sub_graph[0] not in i:
                    # example:
                    # sub_graph = [... a]
                    #             i = [a ...]
                    sub_graph = sub_graph[: len(sub_graph) - 1] + i
                    res.remove(i)
                    res = combine(res, sub_graph)
                check = True
            if sub_graph[0] == i[len(i) - 1]:
                # example:
                # sub_graph = [a ...]
                #     i = [... a]
                sub_graph = i + sub_graph[1 :]
                res.remove(i)
                res = combine(res, sub_graph)
                check = True
        if check == False:
            res.append(sub_graph)
    return res

In [5]:
# this function is creating a result graph containing all maximum contigs
# basically, it pick the largest overlap value of (v, u) as a sub-graph and add it into result
# also, reset (v, u) and (u, v) = 0
# if the termination u existed, ignore that and set (v, u) = 0
def genome_assembly(graph):
    res = [] # result graph
    nod = [] # record which col node has optimal edge
    row, col, value = find_max(graph)
    while value != 0: # when graph still have overlap
        if col not in nod:
            nod.append(col)
            sub_graph = [row, col] # enter a new graph from row to col
            res = combine(res, sub_graph) # combine the sub_graph into result
            graph[row][col] = 0 # set the maximum edge = 0
            graph[col][row] = 0 # self loop = 0
        else: # if col node already has optimal edge
            graph[row][col] = 0 # set the maximum edge = 0
        row, col, value = find_max(graph)
    return res

In [11]:
# this function returns the contigs in order by length
def contigs(res, graph):
    c = [] # contigs
    # calculating all the overlap
    overlap = []
    for i in res:
        o = 0
        for j in range(len(i) - 1):
            o += graph[i[j]][i[j + 1]]
        overlap.append(o)
    # set of start
    # because it might has repeated loop
    # we need to ignor the small overlap one
    s = []
    while(overlap != []):
        # find the maximum overlap
        x = overlap.index(max(overlap))
        check = False
        for i in res[x]:
            if i in s:
                check = True
        if check == False:
            s.append(res[x][0])
            c.append(res[x])
        overlap.remove(overlap[x])
        res.remove(res[x])
    return c

In [22]:
# this function returns the n50 value
def N50_Calculator(c, graph):
    length = []
    for i in c:
        l = 500
        for j in range(len(i) - 1):
            l -= graph[i[j]][i[j + 1]]
            l += 500
        length.append(l)
    half = sum(length) / 2
    l = length
    s, i = 0, 0
    while(s < half):
        s += max(l)
        i = length.index(max(l))
        l.remove(max(l))
    return length[i]

In [21]:
graph, reads = graph_generator("example.txt")
tem_graph = graph
res = genome_assembly(tem_graph)
c = contigs(res, graph)
for i in c:
    print(i)

[69, 70]
[41, 83, 5]
[34, 63, 74]
[88, 64]
[2, 93]
[20, 7, 52, 99, 14, 46]
[27, 89]
[33, 85, 9, 48]
[51, 49, 53]
[54, 31]
[86, 55, 25, 11, 57]
[73, 94, 35]
[87, 17, 6, 39]
[0, 67, 61, 97, 22]
[83, 1, 45]
[1, 59]
[7, 38]
[10, 92, 15, 21]
[15, 77]
[36, 65]


In [23]:
n50 = N50_Calculator(c, graph)
print("100 reads (length 50000) has N50:", n50)

100 reads (length 50000) has N50: 1500
